### C S 329E HW 4

## Pair Programming Group Number: FILL IN HERE
## Members of Team: FILL IN HERE

## Decision Tree Classifier


For this weeks homework we are going to explore ideas around decision tree implementation!  

We will implement some helper functions that would be necessary for a home-grown tree:
  - calc_entropy
  - calc_gini
  
and them test them out at given data splits. 
  
And finally, to perform predictive and descriptive analytics we use the [Decision Tree Classifier](https://scikit-learn.org/stable/modules/tree.html#classification) class in the scikit-learn package.

  
For this assignment, the stopping condition will be the depth of the tree. The impurity measure can be either `Entropy` or `Gini`.

To test our tree built from the Decision Tree Classifier class, we will revisit our Melbourne housing data (that has been cleaned and pruned) and use the files:

   - `melb_tree_train.csv` for training the decision tree (we'll also see what happens if we use the same data to test as we used to train the data in the last problem)
   - `melb_tree_test.csv` for testing the decision tree

There are 10 features in these dataframes that we can use to describe and predict the class label housing "Type", which is 'h' house, 'u' duplex, or 't' townhome. 

In [14]:
# import the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from math import log2
from sklearn import tree # you'll probably need to install this - look at Q6 for a link 
import graphviz # you'll probably need to install this - look at Q6 for a link 

## Q1 Load the Data
Load in the melb_tree_train.csv into a dataframe, and split that dataframe into `df_X`, which contains the features of the data set (everything but `Type`), and `s_y`, the series containing just the class label (just `Type`). The lengths of `df_X` and `s_y` should match.

In [2]:
df = pd.read_csv("melb_tree_train.csv")
df_X = df.drop("Type",1)
#s_y = df["Type"]
s_y= pd.DataFrame(df["Type"], columns = ["Type"]) 


## Q2 Implement a function to calculate entropy 
Implement a function `calc_entropy` that takes the the class label series, `s_y`, as a parameter. Implement using the definition on p128 in the DM book and only use pandas and log2 libraries

In [3]:
# calc_entropy(s_y) definition

def calc_entropy(s_y):
    n = len(s_y)
    entropy = 0;
    for i in s_y.Type.unique():
        ni = int(s_y[s_y["Type"] == i].count())
        if ni ==0:
            continue
        p = ni/n
        entropy += -p * log2(p)
    return entropy

## Q3 Use the entropy function to
  - (a) Calculate the entropy of the entire training set
  - (b) Calculate the entropy of the three partitions formed from 
    * Landsize $\in$ [0,200]
    * Landsize $\in$ (200,450]
    * Landsize $\in$ (450, $\infty$)

In [4]:
# The entire data set
calc_entropy(s_y)

1.5825333114261775

In [5]:
# Less than or equal to 200
less200 = pd.DataFrame(df[df["Landsize"]<=200]["Type"])
calc_entropy(less200)


1.3456432116206725

In [6]:
# Between 200 and 450
btwn200450 = pd.DataFrame(df[(df["Landsize"]>200) & (df["Landsize"]<=450)]["Type"])
calc_entropy(btwn200450)


1.4660501816027978

In [7]:
# greater than 450
grt450 = pd.DataFrame(df[df["Landsize"]>450]["Type"])
calc_entropy(grt450)


1.09954792005911

## Q4 Implement a function to calculate the Gini Index
Implement the function `calc_gini` that takes the class label series, `s_y`, as a parameter. Implement using the definition on p128 in the DM book and only use the pandas library.

In [8]:
def calc_gini( s_y ):
    n = len(s_y)
    gini = 1;
    for i in s_y.Type.unique():
        ni = int(s_y[s_y["Type"] == i].count())
        if ni ==0:
            continue
        p = ni/n
        gini -=p**2
    return gini

## Q5 Use the Gini Index function to
  - (a) Calculate the Gini index of the entire training set
  - (b) Calculate the Gini index of the three partitions formed from 
    * Landsize $\in$ [0,200]
    * Landsize $\in$ (200,450]
    * Landsize $\in$ (450, $\infty$)

In [9]:
# The entire data set
calc_gini(s_y)

0.6655601280292637

In [10]:
# Less than or equal to 200
calc_gini(less200)

0.5741211482422965

In [11]:
# Between 200 and 450
calc_gini(btwn200450)

0.6101021682602731

In [12]:
# greater than 450
calc_gini(grt450)

0.4381934171597633

## Q6 Create a decision tree 
Using [scikit-learn](https://scikit-learn.org/stable/modules/tree.html#tree) create a multi class classifer for the data set using the Entropy impurity measure and a max depth of 3.

Note that scikit-learn's algorithm doesn't handle categorical data, so that needs to be preprocessed using an one hot encoding.

Display the tree using `export_text` from sklearn.tree, and use that information to write some descriptive analytics on the classification of houses.  For extra fun, use the export_graphviz to draw the graph (see documentation on the [scikit-learn webpage](https://scikit-learn.org/stable/modules/tree.html#classification)). 

In [17]:
# Create Tree
ohe = pd.get_dummies(df_X["CouncilArea"])
df_Xohe= pd.concat([df_X.drop("CouncilArea",1),ohe],axis=1)

s_yohe = pd.get_dummies(s_y)
s_y
clf = tree.DecisionTreeClassifier(max_depth=3,criterion="entropy")
clf = clf.fit(df_Xohe, s_yohe)

[Text(167.4, 190.26, 'X[7] <= 99.5\nentropy = 0.917\nsamples = 810\nvalue = [[562, 248]\n[529, 281]\n[529, 281]]'),
 Text(83.7, 135.9, 'X[6] <= 76.0\nentropy = 0.666\nsamples = 291\nvalue = [[257, 34]\n[245, 46]\n[80, 211]]'),
 Text(41.85, 81.53999999999999, 'X[7] <= 81.5\nentropy = 0.258\nsamples = 141\nvalue = [[140, 1]\n[132, 9]\n[10, 131]]'),
 Text(20.925, 27.180000000000007, 'entropy = 0.128\nsamples = 102\nvalue = [[102, 0]\n[99, 3]\n[3, 99]]'),
 Text(62.775000000000006, 27.180000000000007, 'entropy = 0.49\nsamples = 39\nvalue = [[38, 1]\n[33, 6]\n[7, 32]]'),
 Text(125.55000000000001, 81.53999999999999, 'X[8] <= 1957.5\nentropy = 0.854\nsamples = 150\nvalue = [[117, 33]\n[113, 37]\n[70, 80]]'),
 Text(104.625, 27.180000000000007, 'entropy = 0.693\nsamples = 36\nvalue = [[12, 24]\n[34, 2]\n[26, 10]]'),
 Text(146.475, 27.180000000000007, 'entropy = 0.75\nsamples = 114\nvalue = [[105, 9]\n[79, 35]\n[44, 70]]'),
 Text(251.10000000000002, 135.9, 'X[8] <= 1976.5\nentropy = 0.847\nsample

In [25]:
# Display text version of the tree
tree.plot_tree(clf,fontsize=10)


In [ ]:
# Display graphviz version of the tree

➡️ Answer containing your descriptive analytics in markdown here ⬅️

## Q7 Calculate the Accuracy and Display Learning Curve
Load in the test data from melb_tree_test.csv.

Use the scikit-learn library to create many decision trees, each one with a different configuration (aka Hyperparameters).  You will create 28 different trees by:

    - Varying the max depth from 2 to 15 with the Gini Index as the impurity measure
    - Varying the max depth from 2 to 15 with the Entropy as the impurity measure

Implementation tip: you can create an array of numbers from 2 to 15 by using the numpy function [arange](https://numpy.org/doc/stable/reference/generated/numpy.arange.html).

For each of the 28 decistion trees, calculate the error rate by using the data in the: 
  - Training set, and
  - Test set.

Display the results graphicaly, and offer an analysis of the trend (or if no trend present, offer a hypotheisis of why).  The max depth should be on the x-axis, and the error rate should be on the y-axis (see figure 3.23 in your DM textbook for a similar style of graph that uses leaf nodes instead of depth for the x-axis). Your plot will include 4 series of data
   - Test error (entropy)
   - Test error (gini index)
   - Training error (entropy)
   - Training error (gini index)

In [ ]:
# Load in the test data

In [ ]:
# Build the trees using the training data

In [ ]:
# Plot the 4 learning curves

➡️ Answer containing your analysis of the trend (or if no trend present, offer a hypotheisis of why) here ⬅️